# Notebook 2: HOLC and Tree Canopy Mapping
### ESPM C167/Public Health C160 with Professor Rachel Morello-Frosch
**Notebook Developed by Alexander Cui & Ashley Quiterio**

#### Welcome! This activity will take approximately 60 minutes. We hope that by the end of the activity, you will be able to
- Conceptualize HOLC data and understand its current social implications
- Develop specific methods to help read maps and graphs
- Become familiar with tree coverage (tree canopy) data in California 
- Compare the similarities/differences between HOLC and tree canopy data

**Before we get started, let's import the libraries and packages needed for this activity. Don't worry if you don't understand it, just run the cell once prior to starting! Make sure to run this cell every time you open the link.**

In [ ]:
# Tells the notebook not to display warnings
import warnings
warnings.filterwarnings('ignore')
from IPython.display import IFrame

# Libraries that this notebook will use
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from datascience import *
from matplotlib.colors import ListedColormap
import re;

# Helps the maps display nicely in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = [30, 20]

# Color scheming
from matplotlib.colors import ListedColormap
holc_cmap = ListedColormap(['green', 'dodgerblue', 'yellow', 'red'], name='holc_colors');

In [ ]:
# Libraries for map overlays
import contextily as ctx

## 1. Background: *Richmond, Virginia*
<img src="images/tree_canopy_virginia.jpeg" width="550"/>

<!-- BEGIN QUESTION --> 
Let's explore the above photo, which details the neighborhoods of Richmond, Virginia, and their tree coverage. We adapted this photo from the recent New York Times article titled <a href="https://www.nytimes.com/interactive/2020/08/24/climate/racism-redlining-cities-global-warming.html">*How Decades of Racist Housing Policy Left Neighborhoods Sweltering*</a>, published in August of 2020. Financial institutions have often denied redlined areas, highlighted in the image above, banking and mortgage services such as *loans* to purchase homes.

<div class="alert alert-warning">
<b>Warm-Up Question: </b> What patterns do you notice? Why do you think these patterns occurs? How do redlined neighborhoods relate to racist housing policies and the environment? 
Hint: Take a look at the areas surrounding downtown and compare their tree coverage to that of the outer layer of suburbs.
</div>


***Write your answer here, replacing this text.***

<!-- END QUESTION --> 
To understand these questions further, we need to investigate the source of redlining's history: the **Home Owners' Loan Corporation (HOLC)**.

## 2. Home Owners' Loan Corporation (HOLC)

The *Home Owners' Loan Corporation*, otherwise known as **HOLC**, began in the 1930s as a government-financed byproduct of Roosevelt's New Deal. Its original goals were to refinance property mortgages to prevent foreclosures while expanding buying opportunities for those looking for new homes. 

HOLC primarily operated by issuing loans to borrowers, particularly homeowners who struggled to make consistent housing payments. In addition to distributing loans, HOLC also assessed "creditworthiness" for different groups of people within a specific city. In assessing "creditworthiness", HOLC created borders that separated neighborhoods  and targeted minorities (particularly African Americans) on the basis of perceived financial risk. This process of separation became known as ***redlining***, which you can see on *HOLC maps* including the one from the New York Times article above. Let's conceptualize redlining and HOLC maps somewhere closer to home, like Oakland.

<img src="images/oakland_holc_old.jpg" width="600"/>

This is one of the original HOLC maps for the Oakland area. Click [here](https://www.kqed.org/news/11648307/has-oaklands-fruitvale-neighborhood-recovered-from-redlining) to learn more about the history of this map. We will take a closer look at the different parts in the next section when we recreate the map with code.

### 2.1 Understanding HOLC Maps

In HOLC maps, neighborhoods are represented with shapes that enclose the area within a neighborhood. The maps give letter grade ratings to each neighborhood, signifying the neighborhood's investment credibility for lending purposes. Each neighborhood's rating is primarily a function of neighborhood racial and ethnic makeup, housing conditions, and access to public facilities, among other factors. 

Ratings assigned to neighborhoods included: **“A–Best”, “B–Still Desirable”, “C–Definitely Declining”, and “D–Hazardous”.** 

Let's analyze the HOLC map for Oakland.

In [ ]:
# Read in data
oakland = gpd.read_file('holc_oakland')
oakland_converted = oakland.to_crs(epsg=3857)

# Oakland HOLC map
base = oakland_converted.plot(column = 'holc_grade', cmap = holc_cmap, alpha=0.4, legend = True, figsize=[18, 10])
plt.title('Oakland HOLC Grades', size='x-large')
plt.axis('off');

ctx.add_basemap(base, url=ctx.providers.Stamen.TonerLite)

<!-- BEGIN QUESTION -->

<div class="alert alert-warning">
<b>Question 2.1.1: </b> Locate Berkeley on the map above. What do you notice about UC Berkeley? What grades do areas in the city of Berkeley have (north side, south side, downtown), and why do you think this is the case? What factors may separate these areas from the rest? 
</div>


***Write your answer here, replacing this text.***

<!-- END QUESTION --> 

## 3. Connections Between HOLC and Tree Canopy Data

Tree canopy refers to areas that are shaded by trees. To represent this information with data, people abstract colors in an image into squares. You can see this abstraction in the images below. The image on the left is what the ground looks like from above as found on satellite services such as Google Earth. The middle image is a high data version where each square represents a small piece of land in the original photo. The right image is the low data version, which means that each square represents a larger piece of land from the original photo.

To contextualize this, consider the tent/building in the middle of the photo on the left. Since this building itself is not green but its surroundings are green, notice how when the photo is abstracted into squares, the corresponding piece of land remains white. These *squares* and their shades of green are what ultimately determine a square's tree coverage percentage, an idea explained in detail later.


<img src="images/tree_raster.jpg" width="550"/>

[Source](https://spatialvision.com.au/blog-raster-and-vector-data-in-gis/)

Let's think back to the last example. Here is a zoomed-in version of the map from section 2, which captures the Berkeley area. This map is made up of many polygons colored based on the HOLC grade they represent. Each of these shapes is made up of smaller squares such as those in the image above. 

We have selected a smaller portion of the North Berkeley area to go more in depth about how tree canopy is represented. We have to choose a smaller area for our example because there are over 27,000 squares in the Berkeley area alone. For the sake of our example, let's say this area we have selected in the black box below is made up of 16 squares.

<img src="images/berkeley_subset_updated.png" width="550"/>

This area we have selected is made up of squares, and each square represents the land from a satellite perspective and is represented by a shade of either black, white, or green. On the selected area on the most right, we can see what these squares would look like for our example. However, there is another layer of information here. 

Each square corresponds to a percentage of tree coverage. This percentage is used to calculate the average tree coverage across the entire polygon. In our example, we only use the selected section, but the math is generalizable.

<img src="images/berkeley_raster_polygon_example.png" width="550"/>

By averaging the tree coverage percentage across all the squares within a polygon, we can compare the tree coverage over that area with the HOLC grade of that area.

<div class="alert alert-warning">
<b>Question 3.0.1: </b> Describe how the average percentage of tree coverage for the bottom left polygon is calculated.
</div>

***Write your answer here, replacing this text.***

### 3.1 Tree Canopy Data in California Cities

The data we are using is from the [National Land Cover Database](https://www.mrlc.gov/data/nlcd-2016-usfs-tree-canopy-cover-conus). It contains the mean tree canopy rate in 2016 for 7 areas in California (San Francisco, Oakland, San Jose, Fresno, Stockton, Los Angeles, and San Diego). Each city is broken into different neighborhoods based on HOLC areas. 

Note that below, each HOLC graded area corresponds to a polygon (whose shape is described in the `geometry` column). For this reason, polygons and HOLC graded areas can be used interchangeably. Additionally, the phrase *tree coverage percentage* describes a given square's tree coverage expressed as a percent (bounded by 0% [no tree coverage] and 100% [complete tree coverage]). 

*More information on what our dataset contains:*

|Column Name| Meaning|
|---|---|
|holc_grade| HOLC letter grade for a certain neighborhood in a California area|
|area| California City corresponding to a given HOLC graded area |
|count_squares| Total number of squares that are at least 50% covered by a HOLC graded area's borders |
|sum_treeCov| Sum of tree-coverage percentage across all squares in a HOLC graded area|
|avg_treeCov| _sum/_count; Average tree-coverage percentage across a HOLC graded area |
|geometry| Shape of the neighborhood that we use to create our map; encloses a HOLC graded area|

In [ ]:
# Import data
holc_5072 = gpd.read_file('holc_ca_epsg5072_treecov')
holc_5072 = holc_5072.drop(columns=['name', 'holc_id', 'area_descr'])
holc_5072 = holc_5072.rename(columns = {"_count" : "count_squares", "_sum" : "sum_treeCov", 
                             "_mean" : "avg_treeCov"})
holc_5072 = holc_5072[["holc_grade", "area", "count_squares", "sum_treeCov", "avg_treeCov", 
                       "geometry"]]
holc_5072.head()

<!-- BEGIN QUESTION -->

<div class="alert alert-warning">
<b>Question 3.1.1: </b> Look at the first row in the table. What does a tree canopy `count_squares` of 7737 indicate for Oakland? What does a tree canopy `sum_treeCov` of 22.814 indicate? Are these significant given that this area is in an A–HOLC grade area?
</div>

***Write your answer here, replacing this text.***

<!-- END QUESTION --> 
Now that we have a better understanding of HOLC areas and their histories, let's apply this knowledge to a dataset about tree canopy coverage to investigate disparities in these environments. Let's compare these mean tree canopy scores across different HOLC grades. What possible relationship might there be?

### 3.2 HOLC and Tree Canopy Coverage

Below, we created groups where each represent a different HOLC grade. For each group, we introduce some new variables described below.

*More information on what our grouped data contains:*

|Column Name| Meaning|
|---|---|
|color | The color of the HOLC graded area |
|count_squares| Total number of squares that are at least 50% covered by the same HOLC graded area's borders |
|grouped_avg_treeCov| Average tree-coverage percentage across all areas with given HOLC grade|
|HOLC_area_pct| Proportion of total area across 7 cities associated with given HOLC grade |


In [ ]:
summed = holc_5072.groupby('holc_grade').sum()
summed['grouped_avg_treeCov'] = summed['avg_treeCov'] / summed['count_squares']
summed['HOLC_area_pct'] = summed['count_squares'] / sum(summed['count_squares'])
summed['color'] = ['green', 'dodgerblue', 'yellow', 'red']
summed[["color", "count_squares", "grouped_avg_treeCov", "HOLC_area_pct"]]

In [ ]:
plt.rcParams.update({'font.size': 22})

fig = plt.figure()

ax1=plt.subplot(2, 2, 1)
ax1.set_xlabel("Proportion of Total Area (across all 7 cities)")
plt.barh(summed.index, summed["HOLC_area_pct"], color = summed['color'], alpha=0.6)

ax2=plt.subplot(2, 2, 2)
ax2.set_xlabel("Average Tree Coverage Percentage")
plt.barh(summed.index, summed['grouped_avg_treeCov'], color=summed['color'], alpha=0.6);

ax1.set_ylabel('HOLC grade', fontsize='large');

ax1.set_title("Proportion of CA study area by HOLC grade");
ax2.set_title("Average Tree Coverage Percentage by HOLC Grade");

Consider the following to answer the question below:
- Which HOLC grade has the highest average tree coverage percentage? Which has the lowest?
- Which HOLC grades cover the largest proportion of area?

<!-- BEGIN QUESTION -->

<div class="alert alert-warning">
<b>Question 3.2.1: </b> What are the possible implications for neighborhoods in different HOLC areas of different average tree canopy coverage percentages?
</div>

***Write your answer here, replacing this text.***

<!-- END QUESTION -->
We have seen how tree coverage varies across different HOLC grades, but how does this relationship hold for individual cities?

Let's map HOLC areas and tree coverage for different cities in California. 

## 4. Visualizing Tree Canopy and HOLC Data across California

To visualize this, let's play around with a widget that contains the side-by-side comparisons of tree canopy and HOLC Data for Los Angeles, Fresno, San Francisco, San Jose, Stockton, San Diego, and Oakland. Simply select a city in the drop down menu and the widget will update. 
<div class="alert alert-info">
<b>NOTE:</b>  The widget might take about 15 seconds to load after a selection is made. Please wait before switching to another city.
</div>

In [ ]:
import ipywidgets as widgets
plt.rcParams.update({'font.size': 15});

%run -i final_widget_script.py
widgets.VBox([widgets.VBox([inp]), out])

<!-- BEGIN QUESTION --> 
<div class="alert alert-warning">
<b>Question 4.1.1: </b> Based on the city you selected, what patterns do you notice from your plot above? (Note: please clearly state the city you chose.)
</div>

***Write your answer here, replacing this text.***

<div class="alert alert-warning">
<b>Question 4.1.2: </b> Select San Francisco from the drop-down menu. Are there any areas where the association between HOLC grade and tree canopy coverage is contradictory? Why do you think this is?
</div>

***Write your answer here, replacing this text.***

<!-- END QUESTION -->
## 5. Reflection

Today, you had an opportunity to investigate HOLC grades and tree canopy data in California. From the graphs constructed above, you learned how such datasets overlap and can illustrate connections between historical redlining and current racial inequality among neighborhoods. 

<!-- BEGIN QUESTION --> 
<div class="alert alert-warning">
<b>Wrap-Up: </b> Given what was covered in this notebook, do you believe that the relationship between HOLC grades and tree canopy data holds for other states? To what extent could social and racial inequalities be explained through the environment? 
</div>

***Write your answer here, replacing this text.***

<div class="alert alert-info">
<b>Feedback: </b> What was one aspect of the notebook that you found enjoyable? What aspect would you change?
</div>

***Write your answer here, replacing this text.***

<!-- END QUESTION -->
## 6. Submitting your Assignment

You will upload this pdf to bCourses to receive credit for your assignment. Follow these steps to produce a pdf version of your notebook:
1. After running the cell below, save your work. Then, double click on jupyterhub icon in the top left to open in a new tab. 
2. Navigate through the folders: espm-c167 -> Lab2. Find your file named "Tree_Canopy_Notebook_2.pdf"
3. Click on the pdf to open, then download.

If you get stuck, you can find more detailed steps [here](https://docs.google.com/document/d/1usEnHS7T0nx78XM8InB2sIW8YTvnfg8SxZozDm3ASmY/edit?usp=sharing) with image references.

In [ ]:
from otter.export import export_notebook
from IPython.display import display, HTML
export_notebook("Tree_Canopy_Notebook_2.ipynb", filtering=True, pagebreaks=True)
display(HTML("Save this notebook, then click <a href='Tree_Canopy_Notebook_2.pdf' download>here</a> to open the pdf."))

**Acknowledgements:**
- Article referenced: <a href="https://www.nytimes.com/interactive/2020/08/24/climate/racism-redlining-cities-global-warming.html">*How Decades of Racist Housing Policy Left Neighborhoods Sweltering*</a>
- Data from the [National Land Cover Database](https://www.mrlc.gov/data/nlcd-2016-usfs-tree-canopy-cover-conus)